In [1]:
# from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

# agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

# agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")


In [2]:
# memory = [user_defined_task]
# while llm_should_continue(memory): # this loop is the multi-step part
#     action = llm_get_next_action(memory) # this is the tool-calling part
#     observations = execute_action(action)
#     memory += [action, observations]
# https://huggingface.co/blog/smolagents

In [3]:
# from typing import Optional
# from smolagents import CodeAgent, HfApiModel, tool

# @tool
# def get_travel_duration(start_location: str, destination_location: str, transportation_mode: Optional[str] = None) -> str:
#     """Gets the travel time between two places.

#     Args:
#         start_location: the place from which you start your ride
#         destination_location: the place of arrival
#         transportation_mode: The transportation mode, in 'driving', 'walking', 'bicycling', or 'transit'. Defaults to 'driving'.
#     """
#     import os   # All imports are placed within the function, to allow for sharing to Hub.
#     import googlemaps
#     from datetime import datetime

#     gmaps = googlemaps.Client(os.getenv("GMAPS_API_KEY"))

#     if transportation_mode is None:
#         transportation_mode = "driving"
#     try:
#         directions_result = gmaps.directions(
#             start_location,
#             destination_location,
#             mode=transportation_mode,
#             departure_time=datetime(2025, 6, 6, 11, 0), # At 11, date far in the future
#         )
#         if len(directions_result) == 0:
#             return "No way found between these places with the required transportation mode."
#         return directions_result[0]["legs"][0]["duration"]["text"]
#     except Exception as e:
#         print(e)
#         return e

# agent = CodeAgent(tools=[get_travel_duration], model=HfApiModel(), additional_authorized_imports=["datetime"])

# agent.run("Can you give me a nice one-day trip around Paris with a few locations and the times? Could be in the city or outside, but should fit in one day. I'm travelling only with a rented bicycle.")


In [4]:
# https://huggingface.co/collections/m-ric/agents-65ba776fbd9e29f771c07d4e

# https://www.anthropic.com/research/building-effective-agents

# https://huggingface.co/docs/smolagents/examples/multiagents

# https://huggingface.co/docs/smolagents/examples/rag

# what is this ??
# https://e2b.dev/



In [5]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, load_tool, tool
import datetime
import requests
import pytz
import yaml
# from tools.final_answer import FinalAnswerTool

c:\Users\y3kma\anaconda3\envs\llm_v2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
@tool
def my_custom_tool(arg1:str, arg2:int)-> str: # it's important to specify the return type
    # Keep this format for the tool description / args description but feel free to modify the tool
    """A tool that does nothing yet 
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

In [7]:
# https://huggingface.co/spaces/agents-course/First_agent_template/tree/main

from typing import Any, Optional
from smolagents.tools import Tool

class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Provides a final answer to the given problem."
    inputs = {'answer': {'type': 'any', 'description': 'The final answer to the problem'}}
    output_type = "any"

    def forward(self, answer: Any) -> Any:
        return answer

    def __init__(self, *args, **kwargs):
        self.is_initialized = False

In [8]:
final_answer = FinalAnswerTool()
model = HfApiModel(
    max_tokens=2096,
    temperature=0.5,
    model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
    custom_role_conversions=None,
)

with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)
    
# We're creating our CodeAgent
agent = CodeAgent(
    model=model,
    tools=[final_answer], # add your tools here (don't remove final_answer)
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

# GradioUI(agent).launch()

In [ ]:
result = agent.run("What is the current time in America/New_York?")
print(result)
# https://huggingface.co/docs/transformers/en/main_classes/agent


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the current time in America/New_York?                                                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
  import pytz                                                                                                      
                                                                                                                   
  # Get the current time in UTC                                                                                    
  utc_now = datetime.datetime.now(pytz.utc)                                                                        
                                                                                                                   
  # Convert the current UTC time to America/New_York time                                                          
  eastern = pytz.timezone('America/New_York')                                                                      
  eastern_time = utc_now.astimezone(eastern)                                                                       
                                                                                                                   
  # Print the current time in America/New_York                                                                     
  print(eastern_time.strftime('%Y-%m-%d %H:%M:%S %Z%z'))                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Warning to user: Code execution failed due to an unauthorized import - Consider passing said import under 
`additional_authorized_imports` when initializing your CodeAgent.

Code execution failed at line 'import pytz' due to: InterpreterError: Import of pytz is not allowed. Authorized 
imports are: ['time', 'collections', 'math', 'itertools', 'datetime', 'queue', 'random', 'stat', 'unicodedata', 
're', 'statistics']

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Get the current time in UTC                                                                                    
  utc_now = datetime.datetime.now(datetime.timezone.utc)                                                           
                                                                                                                   
  # Define the timezone for America/New_York                                                                       
  eastern = datetime.timezone(datetime.timedelta(hours=-5), name="EST")                                            
                                                                                                                   
  # Convert the current UTC time to America/New_York time                                                          
  eastern_time = utc_now.astimezone(eastern)                                                                       
                                                                                                                   
  # Adjust for daylight saving time if necessary                                                                   
  if eastern_time.dst() != datetime.timedelta(0):                                                                  
      eastern_time = eastern_time.replace(tzinfo=datetime.timezone(datetime.timedelta(hours=-4), name="EDT"))      
                                                                                                                   
  # Print the current time in America/New_York                                                                     
  current_time_eastern = eastern_time.strftime('%Y-%m-%d %H:%M:%S %Z%z')                                           
  print(current_time_eastern)                                                                                      
                                                                                                                   
  # Extract the time part                                                                                          
  time_part = eastern_time.strftime('%H:%M:%S')                                                                    
  final_answer(time_part)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
2025-02-22 13:05:00 EDT-0400

Out - Final answer: 13:05:00

[Step 4: Duration 14.52 seconds| Input tokens: 14,941 | Output tokens: 1,202]

13:05:00


In [ ]:
# interesting... 